In [7]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, cumulative_trapezoid
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import scipy.constants as sc
from pathlib import Path
from numpy.linalg import eigvals

temperature = 300  # K

Angle = np.deg2rad(np.array([-14.92, -10.83, 30.79, -30.79, 10.83, 14.92, -14.91, -13.29, -53.16, 53.16, 13.29, 14.91]))
rotation = np.array([0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 2])
labels = {
    1: {'label': 'T-DPP', 'color': 'b'},
    2: {'label': 'T-T', 'color': 'm'},
    3: {'label': 'T-E', 'color': 'c'},
    # 4: {"label": "FT-FT", "color": 'g'},
    # 5: {"label": "ADTDI-FT", "color": 'r'},
}
ris = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
ris_label = {
    1: {'label': 'DPP-RIS', 'color': 'b'},
    # 2: {'label': 'T-RIS', 'color': 'm'},
    }

In [8]:
def read_data(file_name):
    data = np.loadtxt(file_name)
    data = np.reshape(data, (-1, 2))
    if data[:, 0].max() - data[:, 0].min() != 360:
        mirrored = np.column_stack((-data[:, 0] + 360, data[:, 1]))
        combined = np.vstack((data, mirrored))
        combined = np.unique(combined, axis=0)
        return combined[np.argsort(combined[:, 0])]
    else:
        return data[np.argsort(data[:, 0])]


def fit_function(data_label):
    data = read_data(Path(f"{data_label['label']}.txt"))
    fitf = interp1d(data[:, 0],
                    data[:, 1],
                    kind='cubic',
                    fill_value="extrapolate")
    return fitf


def compute_rotation_integrals(fitf, kTval, limit=1000):

    def exp_energy(phi_deg):
        return np.exp(-fitf(phi_deg) / kTval)

    Z = quad(exp_energy, 0, 360, limit=limit)[0]
    m_i = quad(lambda phi_deg: np.cos(np.deg2rad(phi_deg)) * exp_energy(phi_deg), 0, 360, limit=limit)[0] / Z
    s_i = quad(lambda phi_deg: np.sin(np.deg2rad(phi_deg)) * exp_energy(phi_deg), 0, 360, limit=limit)[0] / Z
    return m_i, s_i

def compute_ris_rotation_integrals(angles_deg, energies, temperature):
    """Compute rotation integrals for RIS model using discrete states."""
    kTval = sc.R * temperature / 1000
    angles_rad = np.deg2rad(angles_deg)

    # Boltzmann 权重
    boltzmann_weights = np.exp(-energies / kTval)
    Z = np.sum(boltzmann_weights)

    probabilities = boltzmann_weights / Z
    m_i = np.sum(probabilities * np.cos(angles_rad))
    s_i = np.sum(probabilities * np.sin(angles_rad))
    return m_i, s_i


def load_ris_data(data_label):
    """Load RIS data and return angles and energies"""
    file_name = Path(f"{data_label['label']}.txt")
    data = np.loadtxt(file_name)
    data = np.reshape(data, (-1, 2))
    data = np.unique(data, axis=0)
    return data[:, 0], data[:, 1]


def make_Mmat(all_data, ris_data, Angle_rad, rotation_types, ris_types, temperature):
    kTval = sc.R * temperature / 1000  # in kJ/mol
    M = len(rotation_types)
    A_list = []
    # Cache for rotation integrals (rot_id -> (m_i, s_i))
    integral_cache = {}
    ris_cache = {}
    for i in range(M):
        rot_id = int(rotation_types[i])
        ris_id = int(ris_types[i])
        theta = float(Angle_rad[i])
        if rot_id == 0 and ris_id == 0:
            m_i, s_i = 1.0, 0.0
        elif rot_id != 0:
            if rot_id not in integral_cache:
                fitf = all_data[rot_id]
                m_i, s_i = compute_rotation_integrals(fitf, kTval)
                integral_cache[rot_id] = (m_i, s_i)
            else:
                m_i, s_i = integral_cache[rot_id]
        elif ris_id != 0:
            if ris_id not in ris_cache:
                angles_deg, energies = ris_data[ris_id]
                m_i, s_i = compute_ris_rotation_integrals(angles_deg, energies, temperature)
                ris_cache[ris_id] = (m_i, s_i)
            else:
                m_i, s_i = ris_cache[ris_id]
        S = np.array([[1, 0.0, 0.0], [0.0, m_i, -s_i], [0.0, s_i, m_i]])
        c = np.cos(theta)
        s = np.sin(theta)
        R_z = np.array([[c, -s, 0.0], [s, c, 0.0], [0.0, 0.0, 1]])
        A_list.append(S @ R_z)

    # Multiply all A_i for the repeat unit
    Mmat = np.eye(3)
    for A in A_list:
        Mmat = A @ Mmat
    return Mmat


def compute_correlation(Mmat):
    """Calculates the correlation length"""
    eigs = eigvals(Mmat)
    lambda_max = float(np.max(np.abs(eigs)))
    if lambda_max >= 1.0:
        return np.inf, 1.0
    corr_length = -1.0 / np.log(lambda_max)
    return corr_length, lambda_max

In [9]:
all_data = {}
for key, label_info in labels.items():
    all_data[key] = fit_function(label_info)
ris_data = {}
for key, label_info in ris_label.items():
    ris_data[key] = load_ris_data(label_info)
matrix = make_Mmat(all_data, ris_data, Angle, rotation, ris, temperature)

In [10]:
lp_repeats, lam = compute_correlation(matrix)
# deflection = np.arccos(lam)

print(f"Max eigen value: lambda_max = {lam:.12f}")
print(f"Correlation length = {lp_repeats:.6f}")
# print(f"Deflection angle (in degrees) = {np.rad2deg(deflection):.6f}")

Max eigen value: lambda_max = 0.895481986633
Correlation length = 9.058531
